In [1]:
from IPython.display import HTML
HTML('<style>div.text_cell_render{font-size:130%;}</style>')
%load_ext version_information
%version_information pandas

Software versions
Python 3.7.1 64bit [Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 7.2.0
OS Darwin 18.2.0 x86_64 i386 64bit
pandas 0.23.4
Sun Mar 03 17:32:48 2019 PST

# Goalie Pull Bayes Optimize

 - Save raw HTML from nhl.com

## Get Training Data

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import datetime
import time
from collections import OrderedDict
import glob
from tqdm import tqdm_notebook
from colorama import Fore, Style

### Download HTML

In [3]:
def random_wait(mu) -> float:
    ''' Positive stochastic var with average of mu '''
    return np.random.beta(3, 3) * mu * 2

def init_sess(sess=None):
    if sess is not None:
        sess.close()
    _sess = requests.Session()
    return _sess

def get_page(sess, url, tries=0) -> str:
    try:
        if tries > 3:
            print(f'Scrape failed at URL = {url}')
            return None

        print(f'Requesting HTML for URL = {url}')
        page = sess.get(url)
        print(f'Got {page.status_code} status code')
        
        if page.status_code == 404:
            print('Bad status code, returning no page')
            return None

        if page.status_code not in (200, 404):
            print('Bad status code, waiting 10 seconds and trying again')
            time.sleep(10)
            sess = init_sess(sess)
            return get_page(sess, url, tries+1)

        return page.text

    except Exception as e:
        print(f'Exception: {str(e)}')
        print('Sleeping, then trying again...')
        time.sleep(10)
        sess = init_sess(sess)
        return get_page(sess, url, tries+1)
        
        
def download_game_range(
    url_template,
    seasons,
    games,
    no_break=False
    ) -> None:
    
    root_data_path = '../../data/raw/html'
    if not os.path.exists(root_data_path):
        os.makedirs(root_data_path)
        print(f'Making dirs {root_data_path}')
        
    request_delay = 3

    print(f'Starting data pull at {datetime.datetime.now()}')

    sess = init_sess()
    for season in seasons:
        data_path = os.path.join(root_data_path, season)
        if not os.path.exists(data_path):
            print(f'Making dirs {data_path}')
            os.makedirs(data_path)
        
        for game_num in games: 
            time.sleep(random_wait(request_delay))

            page_text = get_page(
                sess,
                url_template.format(season, game_num)
            )
            if page_text is None:
                if no_break:
                    print('Bad response, trying next page')
                    continue
                print(f'Season = {season}')
                print(f'Max game = {game_num - 1}')
                break

            f_name = os.path.join(data_path, f'{game_num}.html')
            print(f'Writing HTML to file {f_name}')
            with open(f_name, 'w') as f:
                f.write(page_text)
                
        print(f'Done season {season}')
        if season != seasons[-1]:
            print('Waiting 10 minutes...')
            time.sleep(10*60)

    print(f'Ending data pull at {datetime.datetime.now()}')

In [34]:
# url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
# seasons = ['20032004']
# games = list(range(1, 5000))

# download_game_range(url_tempalte, seasons, games)

Making dirs ../data/html
Starting data pull at 2019-02-03 12:14:32.934214
Making dirs ../data/html/20032004
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020001.HTM
Writing HTML to file ../data/html/20032004/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020002.HTM
Writing HTML to file ../data/html/20032004/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020003.HTM
Writing HTML to file ../data/html/20032004/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020004.HTM
Writing HTML to file ../data/html/20032004/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020005.HTM
Writing HTML to file ../data/html/20032004/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020006.HTM
Writing HTML to file ../data/html/20032004/6.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL0

NameError: name 'download_page' is not defined

 - For 2003/2004 we got up to http://www.nhl.com/scores/htmlreports/20032004/PL021231.HTM

In [5]:
# url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
# seasons = ['20052006']
# games = list(range(1, 5000))

# download_game_range(url_tempalte, seasons, games)

Starting data pull at 2019-02-17 08:25:27.403023
Making dirs ../data/html/20052006
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020001.HTM
Writing HTML to file ../data/html/20052006/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020002.HTM
Writing HTML to file ../data/html/20052006/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020003.HTM
Writing HTML to file ../data/html/20052006/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020004.HTM
Writing HTML to file ../data/html/20052006/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020005.HTM
Writing HTML to file ../data/html/20052006/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020006.HTM
Writing HTML to file ../data/html/20052006/6.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020007.HTM
Writing HTML to

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [8]:
# url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
# seasons = ['20062007']
# games = list(range(1, 5000))

# download_game_range(url_tempalte, seasons, games)

Starting data pull at 2019-02-17 10:45:25.962775
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020001.HTM
Writing HTML to file ../data/html/20062007/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020002.HTM
Writing HTML to file ../data/html/20062007/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020003.HTM
Writing HTML to file ../data/html/20062007/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020004.HTM
Writing HTML to file ../data/html/20062007/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020005.HTM
Writing HTML to file ../data/html/20062007/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020006.HTM
Writing HTML to file ../data/html/20062007/6.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020007.HTM
Writing HTML to file ../data/html/20062007/7.html

In [16]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20032004']
games = list(range(1, 3000))

download_game_range(url_tempalte, seasons, games, no_break=True)

Starting data pull at 2019-02-17 18:12:07.706123
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020001.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020002.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020003.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020004.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020005.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020006.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/6.html
Requesting HTML

KeyboardInterrupt: 

### Legacy format
Pull the old format games. 

In [18]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20032004']
games = list(range(1, 3000))

download_game_range(url_tempalte, seasons, games, no_break=True)

Starting data pull at 2019-02-17 20:34:58.883878
Making dirs ../data/html/20032004
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020001.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020002.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020003.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020004.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020005.HTM
Got 200 status code
Writing HTML to file ../data/html/20032004/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020006.HTM
Got 200 status code
Writing HTML to file ../data/ht

In [20]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20042005']
games = list(range(1, 3000))

download_game_range(url_tempalte, seasons, games, no_break=True)

Starting data pull at 2019-02-17 23:31:05.848611
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20042005/PL020001.HTM
Got 404 status code
Writing HTML to file ../data/html/20042005/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20042005/PL020002.HTM
Got 404 status code
Writing HTML to file ../data/html/20042005/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20042005/PL020003.HTM
Got 404 status code
Writing HTML to file ../data/html/20042005/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20042005/PL020004.HTM
Got 404 status code
Writing HTML to file ../data/html/20042005/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20042005/PL020005.HTM
Got 404 status code
Writing HTML to file ../data/html/20042005/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20042005/PL020006.HTM
Got 404 status code
Writing HTML to file ../data/html/20042005/6.html
Requesting HTML

In [21]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20052006']
games = list(range(1, 3000))

download_game_range(url_tempalte, seasons, games, no_break=True)

Starting data pull at 2019-02-18 02:07:58.687829
Making dirs ../data/html/20052006
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020001.HTM
Got 200 status code
Writing HTML to file ../data/html/20052006/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020002.HTM
Got 200 status code
Writing HTML to file ../data/html/20052006/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020003.HTM
Got 200 status code
Writing HTML to file ../data/html/20052006/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020004.HTM
Got 200 status code
Writing HTML to file ../data/html/20052006/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020005.HTM
Got 200 status code
Writing HTML to file ../data/html/20052006/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20052006/PL020006.HTM
Got 200 status code
Writing HTML to file ../data/ht

In [22]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20062007']
games = list(range(1, 3000))

download_game_range(url_tempalte, seasons, games, no_break=True)

Starting data pull at 2019-02-18 04:45:45.279465
Making dirs ../data/html/20062007
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020001.HTM
Got 200 status code
Writing HTML to file ../data/html/20062007/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020002.HTM
Got 200 status code
Writing HTML to file ../data/html/20062007/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020003.HTM
Got 200 status code
Writing HTML to file ../data/html/20062007/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020004.HTM
Got 200 status code
Writing HTML to file ../data/html/20062007/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020005.HTM
Got 200 status code
Writing HTML to file ../data/html/20062007/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20062007/PL020006.HTM
Got 200 status code
Writing HTML to file ../data/ht

We accidentaly saved all the 404 pages... not sure why as these should have been skipped. Let's clean them up

In [28]:
def clean_folder_404s(season):
    f_pattern = '../../data/raw/html/{}/*.html'.format(season)
    files = sorted(glob.glob(f_pattern))
    print(f'Found {len(files)} files')
    for file in files:
        with open(file, 'r') as f:
            text = f.read()
        if not text.strip():
            print(f'Deleting {file}')
            os.remove(file)

clean_folder_404s('20032004')
clean_folder_404s('20042005')
clean_folder_404s('20052006')
clean_folder_404s('20062007')

Found 2999 files
Deleting ../data/html/20032004/1231.html
Deleting ../data/html/20032004/1232.html
Deleting ../data/html/20032004/1233.html
Deleting ../data/html/20032004/1234.html
Deleting ../data/html/20032004/1235.html
Deleting ../data/html/20032004/1236.html
Deleting ../data/html/20032004/1237.html
Deleting ../data/html/20032004/1238.html
Deleting ../data/html/20032004/1239.html
Deleting ../data/html/20032004/1240.html
Deleting ../data/html/20032004/1241.html
Deleting ../data/html/20032004/1242.html
Deleting ../data/html/20032004/1243.html
Deleting ../data/html/20032004/1244.html
Deleting ../data/html/20032004/1245.html
Deleting ../data/html/20032004/1246.html
Deleting ../data/html/20032004/1247.html
Deleting ../data/html/20032004/1248.html
Deleting ../data/html/20032004/1249.html
Deleting ../data/html/20032004/1250.html
Deleting ../data/html/20032004/1251.html
Deleting ../data/html/20032004/1252.html
Deleting ../data/html/20032004/1253.html
Deleting ../data/html/20032004/1254.html

### Modern format

In [9]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20072008', '20082009', '20092010',
           '20102011', '20112012', '20122013',
           '20132014', '20142015', '20152016',
          '20162017', '20172018']
games = list(range(1, 3000))

download_game_range(url_tempalte, seasons, games)

Starting data pull at 2019-02-17 15:44:14.763811
Making dirs ../data/html/20022003
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20022003/PL020001.HTM
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20022003/PL020001.HTM
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20022003/PL020001.HTM


KeyboardInterrupt: 

Pulled this data using `src/html_download/app.py`